In [11]:
import contextlib
import pathlib

import np_config
import np_session

def size(path: pathlib.Path) -> int:
    return sum(
        f.stat().st_size
        for f in path.rglob('*')
        if f.is_file()
    ) if path.is_dir() else path.stat().st_size
    
def sizes_match(p1, p2):
    return size(p1) == size(p2)

def unlink(path):
    if path.is_dir():
        while path.exists():
            for _ in path.iterdir():
                with contextlib.suppress(OSError):
                    unlink(_)
            path.rmdir()
    else:
        path.unlink()
    
total_bytes = 0  
for _ in np_session.INCOMING_ROOT.iterdir():
    
    try:
        session = np_session.Session(_)
    except np_session.SessionError:
        continue
    
    if not session.lims_path:
        continue
    
    if (
        (existing := (session.lims_path / _.name)).exists()
        and sizes_match(existing, _)
    ):
        total_bytes += size(_)
        unlink(_)
        print(f"Cleared {total_bytes/1024**3:,.0f} GB\r", end='')


In [17]:
total_bytes = 0  
for probe in 'ABCDEF':
    for probe_folder in np_session.INCOMING_ROOT.glob(f'*probe{probe}_sorted'):
        
        try:
            session = np_session.Session(probe_folder)
        except np_session.SessionError:
            continue
        
        if not session.lims_path:
            continue
        
        for path in probe_folder.rglob('*'):
            if path.is_dir():
                continue
            if (existing := tuple(session.lims_path.rglob(f'*/*_probe{probe}/{path.relative_to(probe_folder)}') )):
                for _ in existing:
                    if sizes_match(_, path) and _.stat().st_mtime == path.stat().st_mtime:
                        total_bytes += size(path)
                        unlink(path)
                        print(f"Cleared {total_bytes/1024**3:,.0f} GB\r", end='')
                        break
                        